In [13]:
import pystan
from pystan import StanModel
import numpy as np
import pickle
import os

model = """
data {
    int N;
    int M;
    real<lower=0>  V[N,M];
}
parameters {
    //real<lower=0> lambda;
    real<lower=0> W[N,2];
    real<lower=0> H[2,M];
    real eps;
}
model {
    eps ~ normal(0,1);
    for (i in 1:N)
        for (j in 1:2)
            W[i,j] ~ exponential(0.5);
        
    for (i in 1:2)
        for (j in 1:M)
            H[i,j] ~ exponential(0.5);
    
    for (i in 1:N)
        for (j in 1:M)
            V[i,j] ~ normal(W[i,1]*H[1,j]+W[i,2]*H[2,j], eps);
}
"""
data = np.matrix([[36, 3, 45, 54], \
                            [4, 34, 23, 31], \
                            [9, 65, 11, 0], \
                            [17, 3, 3, 0], \
                            [0, 14, 7, 4]])

In [22]:
data_dict = {'N': data.shape[0], 'M': data.shape[1] ,'V' : data}

if os.path.exists('model.pkl'):
    fit_model = pickle.load(open('model.pkl', 'rb'))
else:   
    with open('model.pkl', 'wb+') as f:
        fit_model = StanModel(model_code=model)
        pickle.dump(fit_eps_model, f)
    

fit = fit_model.sampling(data=data_dict, iter=1000, chains=1)
#fit_eps = pystan.stan(model_code=model, data=data_dict, iter=1000, chains=1)
print(fit)

Inference for Stan model: anon_model_3af51e32c9a82cadff167620130feac4.
1 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=500.

         mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
W[0,0]   0.38    0.01   0.34   0.02   0.14   0.32   0.53   1.25    500    1.0
W[1,0]   3.64    0.07    1.1   1.84    2.9    3.5   4.26   6.13    255    1.0
W[2,0]   7.35    0.14   2.06   3.77   5.96   7.19   8.54  11.86    206    1.0
W[3,0]   0.48    0.02   0.39   0.02   0.18   0.41    0.7   1.35    500    1.0
W[4,0]   1.51    0.03   0.59    0.6   1.07   1.41   1.85   2.87    358    1.0
W[0,1]   9.56    0.26   2.64   5.59   7.53   9.25  11.05   15.7    105    1.0
W[1,1]   3.86    0.11   1.17   2.03   3.04    3.7   4.61   6.33    118    1.0
W[2,1]   0.47    0.02   0.39 4.8e-3   0.14    0.4   0.68   1.46    259    1.0
W[3,1]   0.91    0.04   0.55   0.14    0.5   0.82   1.26    2.1    172    1.0
W[4,1]   0.61    0.03   0.45   0.0

In [18]:
len(fit.extract()['H'])

500

In [19]:
W = fit.extract(['W'])

fit.plot(W)

ValueError: points have dimension 1, dataset has dimension 500

In [23]:
#sm = pystan.StanModel(model_code=model)

#with open('model.pkl', 'wb') as f:

op = fit_model.optimizing(data=data_dict)
print(op)

OrderedDict([('W', array([[  9.04712329e+00,   3.81444816e-03],
       [  3.88509857e+00,   3.23008255e+00],
       [  4.88722388e-01,   6.49541909e+00],
       [  9.05938447e-01,   2.04974372e-01],
       [  5.79732896e-01,   1.35448826e+00]])), ('H', array([[  3.57591586e+00,   2.98524680e-01,   4.89260178e+00,
          6.14647813e+00],
       [  1.53048708e-01,   9.90446939e+00,   1.23564797e+00,
          2.35808011e-03]])), ('eps', array(3.6941974555566297))])


In [24]:
np.matmul(op['W'], op['H'])

array([[ 32.35233545,   2.73856967,  44.2686848 ,  55.60795444],
       [ 14.38714555,  33.15205158,  22.99948513,  23.88729018],
       [  2.74174564,  64.47957526,  10.41717547,   3.01923819],
       [  3.27093072,   2.30060738,   4.68567222,   5.56881419],
       [  2.28037873,  13.58855207,   4.51007287,   3.56650956]])

In [ ]:
np.mat